### Quickstart - Dicas básicas para entendimento e início de desenvolvimento

**Conectando ao EPM Webserver**

In [1]:
import epmwebapi as epm
import numpy as np
import datetime

##Método para não expor usuário e senha do EPM no código fonte
import getpass
user = input('EPM user:')
password = getpass.getpass("EPM password:")

#crie o objeto de conexão informando os endereços do EPM Webserver(Authentication Port e WEB API Port), usuário e senha.
connection = epm.EpmConnection('http://localhost:44333', 'http://localhost:44332', user, password)


EPM user:sa
EPM password:········


**Leitura de dados brutos(Raw)**



In [ ]:
bvname = 'ADM_Temperature'

bv = connection.getDataObjects([bvname])

end_date = datetime.datetime.now()
ini_date= end_date - datetime.timedelta(minutes=10)
queryPeriod = epm.QueryPeriod(ini_date,end_date)

data = bv[bvname].historyReadRaw(queryPeriod)

print(data)


**Leitura usando agregação de dados (Baseados na norma OPC UA)**
Para informações sobre os tipos de agregações, consulte o Apêndice A do Guia do Usuário: https://github.com/elipsesoftware/epmprocessor/blob/master/guiadousuario/Agregacoes.md

In [ ]:
bvname = 'ADM_Temperature'

bv = connection.getDataObjects([bvname])

end_date = datetime.datetime.now()
ini_date= end_date - datetime.timedelta(hours=1)

queryPeriod = epm.QueryPeriod(ini_date,end_date)

processInterval = datetime.timedelta(minutes=10)
aggDetails = epm.AggregateDetails(processInterval, epm.AggregateType.Interpolative)


data = bv[bvname].historyReadAggregate(aggDetails, queryPeriod)

print(data)

**Escrita de conjuntos de dados históricos no EPM Server**


In [ ]:
import numpy as np

bvname = 'bvname'

bv = connection.getDataObjects([bvname])

#some values for write tests
newvalues = [50,60,30,40,10]
base = datetime.datetime(2018, 1, 1)
newdates = np.array([base + datetime.timedelta(hours=i) for i in range(5)])

# epm ndarray data format.
desc = np.dtype([('Value', '>f8'), ('Timestamp', 'object'), ('Quality', '>i4')])
datatemp = np.empty(len(newvalues), dtype=desc)

#loop to populate the object before send to EPM
i=0
while i < len(newvalues):
    datatemp['Value'][i] = newvalues[i]
    datatemp['Timestamp'][i] = newdates[i]
    datatemp['Quality'][i] = 0
    i = i+1
    
bv[bvname].historyUpdate(datatemp)

**Escrita de valores no EPM Server**

In [ ]:
bvname = 'bvname'

bv = connection.getDataObjects([bvname])

date = datetime.datetime.now()
value = 100
quality = 0 #zero is Good in OPC UA


bv[bvname].write(value, date, quality)


**Filtro de variáveis**

In [ ]:
from epmwebapi.dataobjectattributes import DataObjectAttributes
from epmwebapi.dataobjectsfilter import DataObjectsFilter
from epmwebapi.dataobjectsfilter import DataObjectsFilterType
from epmwebapi.domainfilter import DomainFilter

Filter = DataObjectsFilter(DataObjectsFilterType.BasicVariable, '*Temperature*', None, None, DomainFilter.Continuous)

properties = DataObjectAttributes.Description | DataObjectAttributes.LowLimit | DataObjectAttributes.HighLimit | DataObjectAttributes.EU

query = connection.filterDataObjects(doFilter, properties)

#this query returns an OrderedDict
print(query)


**Escrevendo Anotações**

A estrutura de dados do EPM permite que se escrevam anotações nas variáveis. A anotação consiste em mensagem, estampa de tempo e usuário. Nesse exemplo vamos escrever uma anotação no valor máximo da variável no período consultado.

In [33]:
bvname = 'ADM_Temperature'

bv = connection.getDataObjects([bvname])

end_date = datetime.datetime.now()
ini_date= end_date - datetime.timedelta(minutes=10)
queryPeriod = epm.QueryPeriod(ini_date,end_date)
data = bv[bvname].historyReadRaw(queryPeriod)

max_value = data['Value'].max()

max_index = np.argwhere(data['Value'] == max_value)

print(max_index)

print(data['Timestamp'][max_index][0])

#escrevendo anotação
message = 'valor maximo encontrado no periodo'
timestamp = data['Timestamp'][max_index]
bv[bvname].writeAnnotation(timestamp,message)

ValueError: zero-size array to reduction operation maximum which has no identity

**Consultando anotações**

In [10]:
bvname = 'ADM_Temperature'

bv = connection.getDataObjects([bvname])

end_date = datetime.date(2018, 10, 19)
ini_date = end_date - datetime.timedelta(hours=24)

annotations = bv[bvname].readAnnotations(ini_date,end_date)
print(annotations[0][2])

Valor máximo registrado no dia 18/10/2018.


**Fechando a conexão**

É importante fechar a conexão para liberá-la para outros possíveis clients.

In [34]:
connection.close()